In [72]:
import sklearn.mixture
import pickle
import numpy as np
from sklearn.preprocessing import normalize

In [73]:
AMFCC_path='/home/ubuntu/genderRecognition/AMFCC.pk'
GMM_Path='/home/ubuntu/genderRecognition/GMM.pk'
numc_comp=64

In [74]:
male_model= sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='diag')
female_model=sklearn.mixture.GaussianMixture(n_components=numc_comp, 
                                       covariance_type='diag')

In [4]:
f=open(AMFCC_path,'rb')
[train_set_AMFCC,valid_set_AMFCC,test_set_AMFCC,train_set_y,valid_set_y,test_set_y]=pickle.load(f)
f.close()

In [75]:
train_female_feats=np.concatenate([(train_set_AMFCC[i]-np.mean(train_set_AMFCC[i],axis=0))/np.linalg.norm(train_set_AMFCC[i],axis=0) for i in range(len(train_set_AMFCC))[:len(train_set_AMFCC)/2]],axis=0)
train_male_feats=np.concatenate([(train_set_AMFCC[i]-np.mean(train_set_AMFCC[i],axis=0))/np.linalg.norm(train_set_AMFCC[i],axis=0) for i in range(len(train_set_AMFCC))[len(train_set_AMFCC)/2:]],axis=0)

In [76]:
male_model.fit(train_male_feats)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=64, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [77]:
female_model.fit(train_female_feats)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=64, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [78]:
f=open(GMM_Path,'wb')
pickle.dump((male_model,female_model), f)
f.close()

In [102]:
def GenderPredict(feats,male_model,female_model,mode='score'):
    male=0
    male_score_total=0
    female=0
    female_score_total=0
    feats=(feats-np.mean(feats,axis=0))/np.linalg.norm(feats,axis=0)
    for i in range(len(feats)):
        male_score=male_model.score(feats[i].reshape(1,-1))
        female_score=female_model.score(feats[i].reshape(1,-1))
        if male_score>female_score:
            male_score_total+=male_score
            male+=1
        elif male_score<female_score:
            female_score_total+=female_score
            female+=1
    if mode=='score':
        if male_score_total>female_score_total:
            return 1
        else:
            return 0     
    if mode=='most':
        if male>female:
            return 1
        else:
            return 0     
    

In [94]:
feats=test_set_AMFCC[0][0]
feats=(feats-np.mean(feats,axis=0))/np.linalg.norm(feats,axis=0)

In [99]:
female_model.means_ .shape

(64, 39)

In [95]:
feats

array([ 0.39485051,  0.09142322, -0.32680151,  0.044738  , -0.53104905,
       -0.11041967, -0.51841576,  0.08764355, -0.08547259,  0.18069576,
       -0.05609247, -0.08177609, -0.08102805,  0.04146442,  0.03024958,
        0.01185464,  0.03787618,  0.04684236,  0.04767449,  0.03805714,
        0.04001066,  0.03842932,  0.02498511,  0.04677308,  0.03217017,
        0.03970698,  0.03999329,  0.02966882,  0.04479767,  0.05076087,
        0.03029837,  0.05108041,  0.04507982,  0.03509421,  0.04225352,
        0.03824517,  0.03036418,  0.03737768,  0.04059604])

In [100]:
female_model.score(feats.reshape(1,-1))

-3869.8161652894569

In [101]:
male_model.score(feats.reshape(1,-1))

-9258.3315842068059

In [ ]:
prediction=[]
count=0
for i in range(len(test_set_y)):
    count+=1
    if count % int(len(test_set_y)*0.1)==0:
        print('done %s%%'%int(count*10/(len(test_set_y)*0.1)))
    prediction.append(GenderPredict(test_set_AMFCC[i],male_model,female_model))
comparison=[[prediction[i]==test_set_y[i] for i in range(len(test_set_AMFCC))]]

    

done 10%
done 20%
done 30%

In [ ]:
accuracy=sum(comparison)/len(test_set_y)

In [ ]:
accuracy